In [7]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Load your dataset
df = pd.read_csv('BTC-2021min.csv')

# Feature Engineering
df['open-close'] = df['open'] - df['close']
df['low-high'] = df['low'] - df['high']
df['price_change'] = df['close'].pct_change()  # Percentage price change
df['volume_change'] = df['Volume BTC'].pct_change()  # Percentage volume change

# Add rolling averages (e.g., 7-day and 30-day rolling averages)
df['7_day_avg'] = df['close'].rolling(window=7).mean()
df['30_day_avg'] = df['close'].rolling(window=30).mean()

# Replace infinite values with NaN (to handle division by zero cases)
df.replace([np.inf, -np.inf], np.nan, inplace=True)

# Drop NaN values from rolling windows, percentage changes, and other operations
df.dropna(inplace=True)

# Create the binary target: 1 if the price increases, 0 if it decreases
df['target'] = np.where(df['close'].shift(-1) > df['close'], 1, 0)

# Prepare features and target
features = df[['open-close', 'low-high', '7_day_avg', '30_day_avg', 'price_change', 'volume_change']]
target = df['target']

# Split data into training and test sets
X_train, X_valid, Y_train, Y_valid = train_test_split(features, target, test_size=0.1, random_state=2022)

# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)

# Train a Logistic Regression model
logreg = LogisticRegression(max_iter=1000)
logreg.fit(X_train, Y_train)

# Predicting on the validation set
Y_pred = logreg.predict(X_valid)

# Evaluate the model
accuracy = accuracy_score(Y_valid, Y_pred)
print(f"Validation Accuracy: {accuracy * 100:.2f}%")
print("Confusion Matrix:")
print(confusion_matrix(Y_valid, Y_pred))
print("Classification Report:")
print(classification_report(Y_valid, Y_pred))

# Function to predict whether the Bitcoin price will increase or decrease based on user input
def predict_price_direction(open_price, close_price, low_price, high_price, volume, current_price):
    open_close = open_price - close_price
    low_high = low_price - high_price
    price_change = (close_price - open_price) / open_price
    volume_change = 0  # Set volume change to 0 by default if not available
    if len(volume) > 0 and np.any(volume):  # Avoid division by zero or empty array
        volume_change = (volume[-1] - volume[-2]) / volume[-2]

    last_7_day_avg = np.mean(current_price[-7:])
    last_30_day_avg = np.mean(current_price[-30:])

    # Prepare the input features
    future_features = np.array([[open_close, low_high, last_7_day_avg, last_30_day_avg, price_change, volume_change]])

    # Scale the input features
    future_features_scaled = scaler.transform(future_features)

    # Predict whether the price will increase (1) or decrease (0)
    price_direction = logreg.predict(future_features_scaled)

    return "Increase" if price_direction[0] == 1 else "Decrease"

# Example usage: Input from user
open_price = float(input("Enter today's open price: "))
close_price = float(input("Enter today's close price: "))
low_price = float(input("Enter today's low price: "))
high_price = float(input("Enter today's high price: "))
volume = np.array([float(input(f"Enter volume for day {i+1}: ")) for i in range(2)])

# Assume current_price is the last 30 days of closing prices
current_price = df['close'].values[-30:]

print("open",open_price)
print("close",close_price)
print("high",high_price)
print("low",low_price)
print("volume",volume)

# Predict whether the price will increase or decrease
predicted_direction = predict_price_direction(open_price, close_price, low_price, high_price, volume, current_price)
print(f"Predicted Price Direction: {predicted_direction}")


Validation Accuracy: 84.37%
Confusion Matrix:
[[26177  4221]
 [ 5246 24921]]
Classification Report:
              precision    recall  f1-score   support

           0       0.83      0.86      0.85     30398
           1       0.86      0.83      0.84     30167

    accuracy                           0.84     60565
   macro avg       0.84      0.84      0.84     60565
weighted avg       0.84      0.84      0.84     60565

open 33289.0
close 33299.0
high 33308.0
low 33265.0
volume [46.09 41.08]
Predicted Price Direction: Decrease


c:\Users\user\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
